In [1]:
import numpy as np
import pandas as pd
from krippendorff import krippendorff

In [2]:
all_data = pd.read_csv('data/si630w22-hw3-data.csv')
all_data.sample(5)

,question_id,question_text,reply_id,reply_text,rlen
2684,t3_n5bvh6,"Which car is better, manual or automatic?",gx0bgq5,If you see driving as something you enjoy havi...,1472
3464,t3_nllxss,What can the world do completely without that ...,gzjc0dm,Pot holes in the streets,24
2515,t3_ngccat,"Atheists, what is the reason you don't believe...",gyq31nq,"I don't believe in God, I believe in Jod.",41
1668,t3_n5py4f,What was the most ridiculous reason your date ...,gx2la0s,"It was weird, we had three dates and I thought...",375
4869,t3_nkny0h,"ELI5: if gold bullion is a good investment, wh...",gzdtazf,The best way to make money in any market over ...,1014


In [3]:
test_df = pd.read_csv('data/si630w22-hw3-train.csv')
test_df.group.unique()

array(['group_09', 'group_03', 'group_10', 'group_13', 'group_22',
       'group_01', 'group_02', 'group_20', 'group_04', 'group_12',
       'group_15', 'group_24', 'group_17', 'group_25', 'group_19',
       'group_08', 'group_07', 'group_18', 'group_21', 'group_16',
       'group_23', 'group_05', 'group_14', 'group_11'], dtype=object)

In [4]:
group23 = test_df[test_df.group == 'group_23'].copy()
print(group23.shape)
group23.sample(5, random_state=1)

(608, 4)


,id,annotator_id,rating,group
15238,t3_n7bf3w,user_50,4.0,group_23
15410,t3_nko8fo,user_50,4.0,group_23
15320,t3_nd8ssn,user_50,5.0,group_23
15638,t3_nei3i3,user_51,5.0,group_23
15360,t3_ngoh9z,user_50,4.0,group_23


In [5]:
group23_ids = group23.id.unique()
print(len(group23_ids))

304


In [6]:
merged_group23 = group23.merge(right=all_data, how='inner', left_on='id', right_on='question_id')
print(merged_group23.shape)
print(merged_group23.group.unique())
merged_group23.sample(5, random_state=1)

(608, 9)
['group_23']


,id,annotator_id,rating,group,question_id,question_text,reply_id,reply_text,rlen
65,t3_n4scb5,user_51,4.0,group_23,t3_n4scb5,Do you talk to yourself? To what extent?,gwxbok4,It's not weird at all. I do it mostly when I'm...,161
237,t3_nb2kny,user_51,5.0,group_23,t3_nb2kny,"eli5 : Why do we have ""glitch-like"" moment ? L...",gxx33fx,There's a lot of complex thought that comes fr...,568
147,t3_n85dk8,user_51,4.0,group_23,t3_n85dk8,What’s something you for sure know but always ...,gxgkk2n,"I'm good enough, I'm smart enough, and doggone...",66
465,t3_nkblfm,user_51,5.0,group_23,t3_nkblfm,Can you ask for proof of any truths forever? E...,gzcuffi,It's a perfectly good question! Some epistemol...,841
187,t3_n9msxp,user_51,5.0,group_23,t3_n9msxp,"People who got expelled from school, what did ...",gxoovro,"Dress code violation was the final straw, but ...",369


In [7]:
print(merged_group23.groupby('annotator_id').size())
print(merged_group23.id.duplicated().count())

annotator_id
user_50    304
user_51    304
dtype: int64
608


In [8]:
new_index_test = test_df.copy().set_index('id')
new_index_test.loc[list(group23_ids)].groupby('group').size()

group
group_01     26
group_02     36
group_03     22
group_04     18
group_05     32
group_07     42
group_08     26
group_09     36
group_10     36
group_11     39
group_12     36
group_13     24
group_14     24
group_15     30
group_16     22
group_17     48
group_18     42
group_19     26
group_20     36
group_21     24
group_22     36
group_23    608
group_24     36
group_25     24
dtype: int64

In [9]:
group23_overlap = new_index_test.loc[list(group23_ids)].copy()
group23_overlap_wqs = group23_overlap.merge(right=all_data, how='inner', left_on='id', right_on='question_id')

In [10]:
group23_overlap_wqs.head()

,annotator_id,rating,group,question_id,question_text,reply_id,reply_text,rlen
0,user_15,3.0,group_02,t3_n2c9u0,What is One Item you recommend everyone to have?,gwii5or,Rule 9: Never go anywhere without a knife.,42
1,user_16,3.0,group_02,t3_n2c9u0,What is One Item you recommend everyone to have?,gwii5or,Rule 9: Never go anywhere without a knife.,42
2,user_17,3.0,group_02,t3_n2c9u0,What is One Item you recommend everyone to have?,gwii5or,Rule 9: Never go anywhere without a knife.,42
3,user_50,3.0,group_23,t3_n2c9u0,What is One Item you recommend everyone to have?,gwii5or,Rule 9: Never go anywhere without a knife.,42
4,user_51,3.0,group_23,t3_n2c9u0,What is One Item you recommend everyone to have?,gwii5or,Rule 9: Never go anywhere without a knife.,42


## Inter Annotator Agreement

In [11]:
only23 = group23_overlap_wqs[(group23_overlap_wqs.annotator_id == 'user_50') | (group23_overlap_wqs.annotator_id == 'user_51')].copy()
ratings23_df = only23.pivot_table(index='question_id', columns='annotator_id', values='rating')
print(ratings23_df.corr())

annotator_id   user_50   user_51
annotator_id                    
user_50       1.000000  0.957879
user_51       0.957879  1.000000


In [12]:
rd = np.array(only23.pivot_table(index='question_id', columns='annotator_id', values='rating').T)
print(rd.shape)
vd = np.array([1,2,3,4,5])
print(vd.shape)
vc = krippendorff._reliability_data_to_value_counts(rd, vd)
print(vc.shape)

(2, 304)
(5,)
(304, 5)


In [13]:
print("Krippendorff's alpha for nominal metric: ", krippendorff.alpha(value_counts=vc,level_of_measurement="nominal"))
print("Krippendorff's alpha for interval metric: ", krippendorff.alpha(value_counts=vc))

Krippendorff's alpha for nominal metric:  0.9198968908908557
Krippendorff's alpha for interval metric:  0.9576501779111142


## Class-Wide Annotator Agreement

In [14]:
rd_all = np.array(group23_overlap_wqs.pivot_table(index='question_id', columns='annotator_id', values='rating').T)
print(rd_all.shape)
vd_all = np.array([1,2,3,4,5])
print(vd_all.shape)
vc_all = krippendorff._reliability_data_to_value_counts(rd_all, vd_all)
print(vc_all.shape)

(59, 304)
(5,)
(304, 5)


In [15]:
print("Krippendorff's alpha for nominal metric: ", krippendorff.alpha(value_counts=vc_all,level_of_measurement="nominal"))
print("Krippendorff's alpha for interval metric: ", krippendorff.alpha(value_counts=vc_all))

Krippendorff's alpha for nominal metric:  0.2953877580177522
Krippendorff's alpha for interval metric:  0.49413246108797215


**No need to do pearson correlation r on classwide data**

In [16]:
avg_ratings_bygroup = group23_overlap_wqs.pivot_table(index='question_id', columns='group', values='rating', aggfunc=np.mean)
groups_corr = pd.DataFrame(avg_ratings_bygroup.corr().group_23).reset_index()
#groups_corr.to_csv('pearson_correlation_all_annotators.csv', index=False)
groups_corr

,group,group_23
0,group_01,0.469881
1,group_02,0.253562
2,group_03,0.041138
3,group_04,-0.473686
4,group_05,0.844626
5,group_07,0.564816
6,group_08,0.201569
7,group_09,0.618990
8,group_10,0.335148
9,group_11,0.590256


In [72]:
avg_ratings = group23_overlap_wqs.pivot_table(index='question_id', columns='group', values='rating', aggfunc=np.mean)
print(f'average correlation across groups = {avg_ratings.corr().group_23.mean()}')

average correlation across groups = 0.49053466369030446


## Disagreement Evaluation

There are 14 question_ids that exist with group23 rating but don't have a rating from other groups

In [60]:
group23_ratings = merged_group23.pivot_table(index='question_id', values='rating', aggfunc=np.mean).rename(columns={'rating': 'g23_rating'}).reset_index()
print(group23_ratings.shape)

no23_overlap = group23_overlap_wqs[group23_overlap_wqs.group != 'group_23']
no23_ratings = no23_overlap.pivot_table(index='question_id', values='rating', aggfunc=np.mean).rename(columns={'rating': 'other_rating'}).reset_index()
print(no23_ratings.shape)

all_ratings = group23_ratings.merge(right=no23_ratings, how='inner', on='question_id')
all_ratings_wqs = all_ratings.merge(right=all_data, how='inner', on='question_id')

all_ratings_wqs['rating_diff'] = all_ratings_wqs.g23_rating - all_ratings_wqs.other_rating
all_ratings_wqs['rating_abs_diff'] = np.absolute(all_ratings_wqs.rating_diff)

(304, 2)
(290, 2)


In [65]:
other_group_ids = no23_overlap.drop_duplicates(subset='question_id')[['question_id','group']]
all_ratings_wqs = all_ratings_wqs.merge(right=other_group_ids, how='inner', on='question_id')

,question_id,g23_rating,other_rating,question_text,reply_id,reply_text,rlen,rating_diff,rating_abs_diff,group
289,t3_npgle3,2.5,3.0,"Your phone battery is your age, what do you do?",h04x02o,Tell people weird eating habits that help me l...,55,-0.5,0.5,group_25
286,t3_npezra,5.0,4.0,"What is a talent or skill that you have, but d...",h04o737,Sewing. I was making these bags for a while an...,116,1.0,1.0,group_19
8,t3_n37hdl,2.5,1.0,How did you conquer the awkwardness of dancing...,gwnyr78,I never did. I avoid nightclubs instead.,40,1.5,1.5,group_13
196,t3_nijrto,5.0,5.0,What do you think people give too much importa...,gz24aec,Calling people good or bad. It was always like...,248,0.0,0.0,group_07
184,t3_nhleh2,5.0,4.0,"Quick question: If logic is normative, then ar...",gyx26ca,"Arguments are normative in the sense that, giv...",823,1.0,1.0,group_14


In [70]:
top10_mad = all_ratings_wqs.sort_values(by='rating_abs_diff', ascending=False).head(10).drop(columns=['rlen', 'rating_diff'])[['question_id', 'g23_rating', 'other_rating', 'group', 'rating_abs_diff', 'question_text', 'reply_text']]
top10_mad.to_csv('Top10MeanAbsDiff_Ratings.csv', index=False, encoding='utf-8')
top10_mad

,question_id,g23_rating,other_rating,group,rating_abs_diff,question_text,reply_text
82,t3_n9938v,5.0,1.0,group_08,4.0,"Eli5 What is glycogenesis, gluconeogenesis and...",Glycogenesis: how your body creates glycogen. ...
174,t3_ngi7ra,5.0,1.0,group_19,4.0,What is something you don’t regret doing even ...,I threw a brick at a baby goose when I was you...
123,t3_nbz8vw,5.0,1.0,group_11,4.0,What is the dumbest “trend” that people have p...,"Sagging pants with your ass uncovered, and you..."
5,t3_n2w06a,5.0,1.0,group_08,4.0,ELI5: Why are free electrons able to move arou...,Electrons are able to move around objects when...
1,t3_n2hebp,4.5,1.0,group_03,3.5,How do I stop feeling lonely and abandoned in ...,"You just got out of a abusive relationship, th..."
121,t3_nbttn8,5.0,2.0,group_08,3.0,If you never had to worry about money again bu...,Being able to pursue your true passion would b...
213,t3_njwgyd,5.0,2.0,group_08,3.0,Autistic brother is becoming violent when he b...,The most difficult thing is for a parent not t...
263,t3_nnngh1,5.0,2.0,group_08,3.0,Is there a limit on what you’re allowed to res...,There are limits. They will depend on the inst...
198,t3_niqmr6,5.0,2.0,group_09,3.0,Midwestern Americans...what’s the deal with po...,"Lived in the Midwest all my lift (29 years), n..."
166,t3_nfeaw0,5.0,2.0,group_03,3.0,I NEED A GOOD CHOCOLATE CAKE RECIPE WHY IS EVE...,This is my favorite [chocolate cake ](https://...
